In [1]:
import pickle
import MySQLdb
import numpy as np
import random
import matplotlib.pyplot as plt
import datetime
import operator as op
import math
import random
import pandas as pd

In [2]:
# Preliminary query for feature data
sql1 = """
    SELECT
        IF(dxt.AliasName = '', 'OTHERDX', dxt.AliasName) DiagnosisName,
        ptc.Sex,
        TIMESTAMPDIFF(YEAR, ptc.DateOfBirth, ap.ActualStartDate) age,
        DAYOFWEEK(ap.ScheduledStartTime) ap_dow,
        HOUR(ap.ActualStartDate) ap_hour,
        MONTH(ap.ScheduledStartTime) ap_month,
        DATE(ap.ScheduledStartTime) ap_date,
        ap.AliasSerNum,
        ap.PatientSerNum,
        ap.ScheduledStartTime,
        ap.AppointmentSerNum,
        ap.ScheduledEndTime,
        IF((ap.ActualEndDate IN ('0000-00-00 00:00:00','1970-01-01 00:00:00'))
            OR (DATE(ap.ActualStartDate) != DATE(ap.ActualEndDate)),
        '0', TIMESTAMPDIFF(MINUTE, ap.ActualStartDate, ap.ActualEndDate)) duration,
        TIMESTAMPDIFF(MINUTE, ap.ScheduledStartTime, ap.ScheduledEndTime) allocated,
        co.CourseSerNum,
        pd.DoctorSerNum,
        pl.PlanSerNum,
        IF(pl.TreatmentOrientation = '', 'NULL', pl.TreatmentOrientation) orientation
    FROM
        Appointment ap
    INNER JOIN Diagnosis dx
        ON dx.DiagnosisSerNum = ap.DiagnosisSerNum
    LEFT JOIN DiagnosisTranslation dxt
        ON dxt.DiagnosisCode = dx.DiagnosisCode
    INNER JOIN Patientcopy ptc
        ON ptc.PatientSerNum = ap.PatientSerNum
    INNER JOIN Course co
        ON co.PatientSerNum = ptc.PatientSerNum
    INNER JOIN Plan pl
        ON pl.CourseSerNum = co.CourseSerNum
    INNER JOIN PatientDoctor pd
        ON pd.PatientSerNum = ap.PatientSerNum
    WHERE
        ap.Status LIKE '%%Completed%%'
    AND ap.State = 'Active'
    AND ap.ActualStartDate NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
    AND pd.PrimaryFlag = 1
    AND pd.OncologistFlag = 1
    AND pl.Status IN ('TreatApproval')
    AND co.CourseSerNum = (
        SELECT co2.CourseSerNum
        FROM Course co2
        INNER JOIN Plan pl2
        ON (    pl2.CourseSerNum = co2.CourseSerNum
            AND pl2.Status IN ('TreatApproval') )
        WHERE
            ap.ActualStartDate  > co2.StartDateTime
        AND co2.CourseId        NOT LIKE '%%QA%%'
        AND co.PatientSerNum    = co2.PatientSerNum
        ORDER BY co2.StartDateTime DESC
        LIMIT 1 )
    AND ap.AliasSerNum IN ('31', '23')
    AND ap.ScheduledStartTime > '2015-01-01 00:00:00'
    GROUP BY
        ptc.PatientSerNum, ap.ScheduledStartTime
    ORDER BY
        ap.AppointmentSerNum ASC
"""

In [3]:
# Connect to the MySQL database
def getSQLtest(sql):
    # Connect to the MySQL database
    db = MySQLdb.connect(host='localhost',
                         port=3306,
                         user='root',
                         passwd='root',
                         db='originaldata')
    db_cursor = db.cursor(MySQLdb.cursors.DictCursor)
    sql0 = """
         set sql_mode = 'STRICT_TRANS_TABLES,NO_ZERO_IN_DATE,NO_ZERO_DATE,ERROR_FOR_DIVISION_BY_ZERO,NO_ENGINE_SUBSTITUTION';
    """       
    db_cursor.execute(sql0)
    db_cursor.execute(sql)
    db_results = db_cursor.fetchall()
    db_cursor.close()
    print ("Fetched Preliminary query results")
    print ("Number of results: " + str(len(db_results)))
    return db_results

In [4]:
def getMedianDuration(patient_ser, appt_date, plan):
    """ Function to get a patient's average past appointment durations.
    Note: given the same plan. """

    med_duration = 0.0
    durations = list()
    areDurations = False
    sql = """
        SELECT
            ap.AppointmentSerNum,
            IF((ap.ActualEndDate IN ('0000-00-00 00:00:00','1970-01-01 00:00:00'))
                OR (DATE(ap.ActualStartDate) != DATE(ap.ActualEndDate)),
            '0', TIMESTAMPDIFF(MINUTE, ap.ActualStartDate, ap.ActualEndDate)) duration
        FROM
            Appointment ap
        INNER JOIN Patient pt
            ON pt.PatientSerNum = ap.PatientSerNum
        INNER JOIN Course co
            ON co.PatientSerNum = pt.PatientSerNum
        INNER JOIN Plan pl
            ON pl.CourseSerNum = co.CourseSerNum
        WHERE
            ap.Status NOT LIKE '%%Cancelled%%'
        AND ap.ActualStartDate NOT IN ('0000-00-00 00:00:00','1970-01-01 00:00:00')
        AND ap.AliasSerNum IN ('31', '23')
        AND pt.PatientSerNum = %s
        AND ap.ScheduledStartTime < '%s'
        AND pl.PlanSerNum = %s
    """ % (str(patient_ser), str(appt_date), str(plan))
    
    print("Executing query...")
    db_results = getSQLtest(sql)

    # Loop through each result
    for row in db_results:
        duration = row['duration']
        if duration == 0:
            continue
        else:
            areDurations = True
            durations.append(float(duration))

    if areDurations:
        med_duration = np.median(durations)

    return med_duration

In [5]:
print("Executing query...")
db_results1 = getSQLtest(sql1)
print('Finish!')

Executing query...
Fetched Preliminary query results
Number of results: 144873
Finish!


In [7]:
print('Start getting raw_patients dictionary')
count = 1
raw_patients = dict()

# Loop through each result
for row in db_results1:

    # In this next instance, there may be duplicate appointment serials for each patient
    # because of multiple plans for each course (or other duplicate info). We must concatenate all duplicates
    # into one list because all of them together contribute to ONE appointment duration.
    # If we didn't do this then we would be saying that each duplicate contributes to
    # it's own duration, which is not accurate. For ex: one appt has FP1_BREAST and FP1_EL.
    # Separately, each of them has an assigned duration (2 query results), let's say 10 minutes.
    # This is ONE appt, so we can't say that each plan is 10 mins long. In fact,
    # BOTH plans together contribute to the 10 minutes.
    
#        在下一个例子中，由于每个疗程有多个计划(或其他重复信息)，每个病人可能有重复的预约序列。
#        我们必须将所有的副本连接到一个列表中，因为所有这些副本一起构成一个约会持续时间。
#        如果我们不这样做，那么我们就会说每个副本都贡献了它自己的持续时间，这是不准确的。
#        例如:一个appt有FP1_BREAST和FP1_EL。另外，它们每个都有一个指定的持续时间(2个查询结果)，比如10分钟。
#        这是一个appt，所以我们不能说每个计划都是10分钟。事实上，这两种计划加在一起就是10分钟。
    
    
    # Retrieve row params
    diagnosis = str(row['DiagnosisName'])
    gender = str(row['Sex'])
    age = int(row['age'])
    appt_day_of_week = int(row['ap_dow'])
    appt_hour = int(row['ap_hour'])
    appt_month = int(row['ap_month'])
    appt_date = str(row['ap_date'])
    alias_ser = int(row['AliasSerNum'])
    patient_ser = int(row['PatientSerNum'])
    appt_ser = str(row['AppointmentSerNum'])
    scheduled_start = str(row['ScheduledStartTime'])
    scheduled_end = str(row['ScheduledEndTime'])
    appt_allocated = int(row['allocated'])
    course = str(row['CourseSerNum'])
    oncologist = str(row['DoctorSerNum'])
    plan = str(row['PlanSerNum'])
    orientation = str(row['orientation'])
    appt_duration = int(row['duration'])
    #avg_duration = getMedianDuration(patient_ser, appt_date, plan)
    
    if count % 1000 == 0:
        print(count)
    count += 1
    
    # If current patient is not in the keys of patients,
    # Add a new key and assign the initial patient entry as defaultdicts
    if patient_ser not in raw_patients.keys():
        raw_patients[patient_ser] = {
                appt_date: {
                    'diagnosis': [diagnosis],
                    'oncologist': [oncologist],
                    'course': [course],
                    'gender': [gender],
                    'age': [age],
                    'appt_day_of_week': [appt_day_of_week],
                    'appt_hour': [appt_hour],
                    'appt_month': [appt_month],
                    'plan': [plan],
                    'orientation': [orientation],
                    #'avg_duration': avg_duration,
                    'appt_allocated': [appt_allocated],
                    'appt_duration': [appt_duration]
                }
        }

        continue

    if appt_date not in raw_patients[patient_ser].keys():
        raw_patients[patient_ser][appt_date] = {
            'diagnosis': [diagnosis],
            'oncologist': [oncologist],
            'course': [course],
            'gender': [gender],
            'age': [age],
            'appt_day_of_week': [appt_day_of_week],
            'appt_hour': [appt_hour],
            'appt_month': [appt_month],
            'plan': [plan],
            'orientation': [orientation],
            #'avg_duration': avg_duration,
            'appt_allocated': [appt_allocated],
            'appt_duration': [appt_duration]
        }

        continue


    # At this point we've reached duplicate patient serials.
    # We append the respective parameters for the same appointment (ignoring duplicate parameters)
    raw_patients[patient_ser][appt_date]['diagnosis']           = list(set(raw_patients[patient_ser][appt_date]['diagnosis'] + [diagnosis]))
    raw_patients[patient_ser][appt_date]['oncologist']          = list(set(raw_patients[patient_ser][appt_date]['oncologist'] + [oncologist]))
    raw_patients[patient_ser][appt_date]['course']              = list(set(raw_patients[patient_ser][appt_date]['course'] + [course]))
    raw_patients[patient_ser][appt_date]['gender']              = list(set(raw_patients[patient_ser][appt_date]['gender'] + [gender]))
    raw_patients[patient_ser][appt_date]['age']                 = list(set(raw_patients[patient_ser][appt_date]['age'] + [age]))
    raw_patients[patient_ser][appt_date]['appt_day_of_week']    = list(set(raw_patients[patient_ser][appt_date]['appt_day_of_week'] + [appt_day_of_week]))
    raw_patients[patient_ser][appt_date]['appt_hour']           = list(set(raw_patients[patient_ser][appt_date]['appt_hour'] + [appt_hour]))
    raw_patients[patient_ser][appt_date]['appt_month']          = list(set(raw_patients[patient_ser][appt_date]['appt_month'] + [appt_month]))
    raw_patients[patient_ser][appt_date]['plan']                = list(set(raw_patients[patient_ser][appt_date]['plan'] + [plan]))
    raw_patients[patient_ser][appt_date]['orientation']         = list(set(raw_patients[patient_ser][appt_date]['orientation'] + [orientation]))
    raw_patients[patient_ser][appt_date]['appt_allocated']      += [appt_allocated]
    raw_patients[patient_ser][appt_date]['appt_duration']       += [appt_duration]

print('Finish getting raw_patients dictionary')

Start getting raw_patients dictionary
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
Finish getting raw_patients dictionary


In [15]:
print(len(raw_patients))
raw_patients.keys()
raw_patients[1054]

9432


{'2015-03-24': {'diagnosis': ['Resp'],
  'oncologist': ['1338'],
  'course': ['39079'],
  'gender': ['Female'],
  'age': [80],
  'appt_day_of_week': [3],
  'appt_hour': [13],
  'appt_month': [3],
  'plan': ['99865'],
  'orientation': ['HFS'],
  'appt_allocated': [15],
  'appt_duration': [19]},
 '2015-03-25': {'diagnosis': ['Resp'],
  'oncologist': ['1338'],
  'course': ['39079'],
  'gender': ['Female'],
  'age': [80],
  'appt_day_of_week': [4],
  'appt_hour': [13],
  'appt_month': [3],
  'plan': ['99865'],
  'orientation': ['HFS'],
  'appt_allocated': [15],
  'appt_duration': [12]},
 '2015-03-26': {'diagnosis': ['Resp'],
  'oncologist': ['1338'],
  'course': ['39079'],
  'gender': ['Female'],
  'age': [80],
  'appt_day_of_week': [5],
  'appt_hour': [13],
  'appt_month': [3],
  'plan': ['99865'],
  'orientation': ['HFS'],
  'appt_allocated': [15],
  'appt_duration': [17]},
 '2015-03-27': {'diagnosis': ['Resp'],
  'oncologist': ['1338'],
  'course': ['39079'],
  'gender': ['Female'],
  '

In [16]:
sql2 = """
    SELECT
        rh.RadiationHstryAriaSer,
        DATE(rh.TreatmentStartTime) tx_date,
        rh.TreatmentStartTime,
        rh.TreatmentEndTime,
        rh.FractionNumber,
        rh.ImagesTaken,
        rh.ImageDuration,
        rh.UserName,
        ra.RadiationSerNum,
        ra.RadiationId,
        ra.ResourceSerNum,
        ra.MU,
        ra.MUCoeff,
        ptc.PatientSerNum,
        co.CourseId
    FROM
        RadiationHstry rh
    INNER JOIN Radiation ra
        ON rh.RadiationSerNum = ra.RadiationSerNum
    INNER JOIN Plan pl
        ON ra.PlanSerNum = pl.PlanSerNum
    INNER JOIN Course co
        ON pl.CourseSerNum = co.CourseSerNum
    INNER JOIN Patientcopy ptc
        ON co.PatientSerNum = ptc.PatientSerNum
    WHERE
        ra.DeliveryType = 'TREATMENT'
    AND ra.MUCoeff > 0
    AND ra.MU > 0
    AND rh.ImagesTaken = (
        SELECT MAX(rh2.ImagesTaken)
        FROM RadiationHstry rh2
        WHERE
            rh2.FractionNumber = rh.FractionNumber
        AND rh2.RadiationSerNum = rh.RadiationSerNum
        AND DATE(rh2.TreatmentStartTime) = DATE(rh.TreatmentStartTime) )
    AND rh.TreatmentStartTime > '2015-01-01 00:00:00'
    GROUP BY
        ptc.PatientSerNum, ra.RadiationId, rh.FractionNumber, DATE(rh.TreatmentStartTime)
    ORDER BY
        rh.FractionNumber, rh.TreatmentStartTime
"""
# AND pt.PatientId REGEXP '^[[:digit:]]+$'

In [17]:
print("Executing query...")
db_results = getSQLtest(sql2)
print('Finish!')

Executing query...
Fetched Preliminary query results
Number of results: 426811
Finish!


In [21]:
print('Start getting raw_treatments dictionary')
count = 1
raw_treatments = dict()
# Loop through each result
    # Loop through each result
for row in db_results:

    # Retrieve query params
    radiationhstry_aria = int(row['RadiationHstryAriaSer'])
    tx_date = str(row['tx_date'])
    tx_start = str(row['TreatmentStartTime'])
    tx_end = str(row['TreatmentEndTime'])
    fraction = int(row['FractionNumber'])
    images_taken = int(row['ImagesTaken'])
    image_duration = int(row['ImageDuration'])
    therapist = str(row['UserName'])
    radiation_ser = int(row['RadiationSerNum'])
    radiation_id = str(row['RadiationId'])
    resource_ser = str(row['ResourceSerNum'])
    MU = float(row['MU'])
    MUCoeff = float(row['MUCoeff'])
    patient_ser = int(row['PatientSerNum'])
    course_id = str(row['CourseId'])


    # Check if the patient appointment exists for this treatment entry
    # If not, continue
    if patient_ser not in raw_patients.keys():
        continue
    elif tx_date not in raw_patients[patient_ser].keys():
        continue

    # continue 跳过当前循环的全部语句，执行下一次循环
    
    # At this point an appointment exists for this treatment entry
    if patient_ser not in raw_treatments.keys():
        raw_treatments[patient_ser] = {
            course_id: {
                fraction: {
                    tx_date: {
                        'radiation_id': [radiation_id],
                        'radiation_ser': [radiation_ser],
                        'num_of_fields': 1,
                        'images_taken': images_taken,
                        #'prev_duration': [0],
                        #'prev_duration': raw_patients[patient_ser][tx_date]['avg_duration'],
                        'image_duration': image_duration,
                        'therapist': [therapist],
                        'tot_mu': MU,
                        'tot_mucoeff': MUCoeff,
                        'resource': resource_ser,
                        #'avg_duration': raw_patients[patient_ser][tx_date]['avg_duration'],
                        'allocated': raw_patients[patient_ser][tx_date]['appt_allocated'],
                        'duration': raw_patients[patient_ser][tx_date]['appt_duration'],
                        'diagnosis': raw_patients[patient_ser][tx_date]['diagnosis'],
                        'gender': raw_patients[patient_ser][tx_date]['gender'],
                        'oncologist': raw_patients[patient_ser][tx_date]['oncologist'],
                        'age': raw_patients[patient_ser][tx_date]['age'],
                        'appt_day_of_week': raw_patients[patient_ser][tx_date]['appt_day_of_week'],
                        'appt_month': raw_patients[patient_ser][tx_date]['appt_month'],
                        'appt_hour': raw_patients[patient_ser][tx_date]['appt_hour'],
                        'plan': raw_patients[patient_ser][tx_date]['plan'],
                        'orientation': raw_patients[patient_ser][tx_date]['orientation']
                    }
                }
            }
        }
        
        if count % 1000 == 0:
            print ("MAIN: " + str(count))
        count += 1
        continue

    if course_id not in raw_treatments[patient_ser].keys():
        raw_treatments[patient_ser][course_id] = {
            fraction: {
                tx_date: {
                    'radiation_id': [radiation_id],
                    'radiation_ser': [radiation_ser],
                    'num_of_fields': 1,
                    'images_taken': images_taken,
                    #'prev_duration': [0],
                    #'prev_duration': raw_patients[patient_ser][tx_date]['avg_duration'],
                    'image_duration': image_duration,
                    'therapist': [therapist],
                    'tot_mu': MU,
                    'tot_mucoeff': MUCoeff,
                    'resource': resource_ser,
                    #'avg_duration': raw_patients[patient_ser][tx_date]['avg_duration'],
                    'allocated': raw_patients[patient_ser][tx_date]['appt_allocated'],
                    'duration': raw_patients[patient_ser][tx_date]['appt_duration'],
                    'diagnosis': raw_patients[patient_ser][tx_date]['diagnosis'],
                    'gender': raw_patients[patient_ser][tx_date]['gender'],
                    'oncologist': raw_patients[patient_ser][tx_date]['oncologist'],
                    'age': raw_patients[patient_ser][tx_date]['age'],
                    'appt_day_of_week': raw_patients[patient_ser][tx_date]['appt_day_of_week'],
                    'appt_month': raw_patients[patient_ser][tx_date]['appt_month'],
                    'appt_hour': raw_patients[patient_ser][tx_date]['appt_hour'],
                    'plan': raw_patients[patient_ser][tx_date]['plan'],
                    'orientation': raw_patients[patient_ser][tx_date]['orientation']
                }
            }
        }

        if count % 1000 == 0:
            print ("MAIN: " + str(count))
        count += 1
        continue

    if fraction not in raw_treatments[patient_ser][course_id].keys():

        #prev_duration = [0]
        #prev_duration = raw_patients[patient_ser][tx_date]['avg_duration']
        
        # get previous fraction
        prev_fraction = sorted(raw_treatments[patient_ser][course_id].keys())[-1]
        # get the previous fraction keys (ie. the tx dates from the previous fraction)
        last_frac_dates = raw_treatments[patient_ser][course_id][prev_fraction].keys()
        # loop through each tx date of the previous fraction to find the right tx date
        # corresponding to the previous treatment
        for last_date in last_frac_dates:
            # the radiation serials for this tx date
            prev_radiation_ser = raw_treatments[patient_ser][course_id][prev_fraction][last_date]['radiation_ser']
            # if the radiation serial from query matches, then this corresponds to the previous tx
            if radiation_ser in prev_radiation_ser:
                # previous duration is the duration from the prev tx date of the previous fraction
                prev_duration = raw_treatments[patient_ser][course_id][prev_fraction][last_date]['duration']
                break

        raw_treatments[patient_ser][course_id][fraction] = {
            tx_date: {
                'radiation_id': [radiation_id],
                'radiation_ser': [radiation_ser],
                'num_of_fields': 1,
                'images_taken': images_taken,
                #'prev_duration': prev_duration,
                'image_duration': image_duration,
                'therapist': [therapist],
                'tot_mu': MU,
                'tot_mucoeff': MUCoeff,
                'resource': resource_ser,
                #'avg_duration': raw_patients[patient_ser][tx_date]['avg_duration'],
                'allocated': raw_patients[patient_ser][tx_date]['appt_allocated'],
                'duration': raw_patients[patient_ser][tx_date]['appt_duration'],
                'diagnosis': raw_patients[patient_ser][tx_date]['diagnosis'],
                'gender': raw_patients[patient_ser][tx_date]['gender'],
                'oncologist': raw_patients[patient_ser][tx_date]['oncologist'],
                'age': raw_patients[patient_ser][tx_date]['age'],
                'appt_day_of_week': raw_patients[patient_ser][tx_date]['appt_day_of_week'],
                'appt_month': raw_patients[patient_ser][tx_date]['appt_month'],
                'appt_hour': raw_patients[patient_ser][tx_date]['appt_hour'],
                'plan': raw_patients[patient_ser][tx_date]['plan'],
                'orientation': raw_patients[patient_ser][tx_date]['orientation']
            }
        }

        if count % 1000 == 0:
            print ("MAIN: " + str(count))
        count += 1
        continue

    if tx_date not in raw_treatments[patient_ser][course_id][fraction].keys():

        #prev_duration = [0]
        #prev_duration = raw_patients[patient_ser][tx_date]['avg_duration']
        if fraction > 1 and len(raw_treatments[patient_ser][course_id].keys()) > 1:

            # get previous fraction. In this case, this fraction exists in the keys
            # so the real previous fraction is the 2nd from the last
            prev_fraction = sorted(raw_treatments[patient_ser][course_id].keys())[-2]
            # get the previous fraction keys (ie. the tx dates from the previous fraction)
            last_frac_dates = raw_treatments[patient_ser][course_id][prev_fraction].keys()
            # loop through each tx date of the previous fraction to find the right tx date
            # corresponding to the previous treatment
            for last_date in last_frac_dates:
                # the radiation serials for this last tx date
                prev_radiation_ser = raw_treatments[patient_ser][course_id][prev_fraction][last_date]['radiation_ser']
                # if the radiation serial from query matches, then this corresponds to the previous tx
                if radiation_ser in prev_radiation_ser:
                    # previous duration is the duration from the prev tx date of the previous fraction
                    prev_duration = raw_treatments[patient_ser][course_id][prev_fraction][last_date]['duration']
                    break

        raw_treatments[patient_ser][course_id][fraction][tx_date] = {
            'radiation_id': [radiation_id],
            'radiation_ser': [radiation_ser],
            'num_of_fields': 1,
            'images_taken': images_taken,
            #'prev_duration': prev_duration,
            'image_duration': image_duration,
            'therapist': [therapist],
            'tot_mu': MU,
            'tot_mucoeff': MUCoeff,
            'resource': resource_ser,
            #'avg_duration': raw_patients[patient_ser][tx_date]['avg_duration'],
            'allocated': raw_patients[patient_ser][tx_date]['appt_allocated'],
            'duration': raw_patients[patient_ser][tx_date]['appt_duration'],
            'diagnosis': raw_patients[patient_ser][tx_date]['diagnosis'],
            'gender': raw_patients[patient_ser][tx_date]['gender'],
            'oncologist': raw_patients[patient_ser][tx_date]['oncologist'],
            'age': raw_patients[patient_ser][tx_date]['age'],
            'appt_day_of_week': raw_patients[patient_ser][tx_date]['appt_day_of_week'],
            'appt_month': raw_patients[patient_ser][tx_date]['appt_month'],
            'appt_hour': raw_patients[patient_ser][tx_date]['appt_hour'],
            'plan': raw_patients[patient_ser][tx_date]['plan'],
            'orientation': raw_patients[patient_ser][tx_date]['orientation']
        }
   
        if count % 1000 == 0:
            print ("MAIN: " + str(count))
        count += 1
        continue


    # At this point we've reached duplicate tx dates for the same fraction, course and patient.
    # We append the respective parameters for the same tx params
    raw_treatments[patient_ser][course_id][fraction][tx_date]['radiation_id']       = list(set(raw_treatments[patient_ser][course_id][fraction][tx_date]['radiation_id'] + [radiation_id]))
    raw_treatments[patient_ser][course_id][fraction][tx_date]['radiation_ser']      = list(set(raw_treatments[patient_ser][course_id][fraction][tx_date]['radiation_ser'] + [radiation_ser]))
    raw_treatments[patient_ser][course_id][fraction][tx_date]['therapist']          = list(set(raw_treatments[patient_ser][course_id][fraction][tx_date]['therapist'] + [therapist]))
    raw_treatments[patient_ser][course_id][fraction][tx_date]['num_of_fields']      = len(raw_treatments[patient_ser][course_id][fraction][tx_date]['radiation_id'])
    raw_treatments[patient_ser][course_id][fraction][tx_date]['image_duration']     = min(raw_treatments[patient_ser][course_id][fraction][tx_date]['image_duration'], image_duration)
    raw_treatments[patient_ser][course_id][fraction][tx_date]['tot_mu']             += MU
    raw_treatments[patient_ser][course_id][fraction][tx_date]['tot_mucoeff']        += MUCoeff

    if count % 1000 == 0:
        print ("MAIN: " + str(count))
    count += 1
print('Start getting raw_treatments dictionary')

Start getting raw_treatments dictionary
MAIN: 1000
MAIN: 2000
MAIN: 3000
MAIN: 4000
MAIN: 5000
MAIN: 6000
MAIN: 7000
MAIN: 8000
MAIN: 9000
MAIN: 10000
MAIN: 11000
MAIN: 12000
MAIN: 13000
MAIN: 14000
MAIN: 15000
MAIN: 16000
MAIN: 17000
MAIN: 18000
MAIN: 19000
MAIN: 20000
MAIN: 21000
MAIN: 22000
MAIN: 23000
MAIN: 24000
MAIN: 25000
MAIN: 26000
MAIN: 27000
MAIN: 28000
MAIN: 29000
MAIN: 30000
MAIN: 31000
MAIN: 32000
MAIN: 33000
MAIN: 34000
MAIN: 35000
MAIN: 36000
MAIN: 37000
MAIN: 38000
MAIN: 39000
MAIN: 40000
MAIN: 41000
MAIN: 42000
MAIN: 43000
MAIN: 44000
MAIN: 45000
MAIN: 46000
MAIN: 47000
MAIN: 48000
MAIN: 49000
MAIN: 50000
MAIN: 51000
MAIN: 52000
MAIN: 53000
MAIN: 54000
MAIN: 55000
MAIN: 56000
MAIN: 57000
MAIN: 58000
MAIN: 59000
MAIN: 60000
MAIN: 61000
MAIN: 62000
MAIN: 63000
MAIN: 64000
MAIN: 65000
MAIN: 66000
MAIN: 67000
MAIN: 68000
MAIN: 69000
MAIN: 70000
MAIN: 71000
MAIN: 72000
MAIN: 73000
MAIN: 74000
MAIN: 75000
MAIN: 76000
MAIN: 77000
MAIN: 78000
MAIN: 79000
MAIN: 80000
MAIN: 810

In [25]:
print(len(raw_treatments))
raw_treatments.keys()
raw_treatments[36466]

8880


{'C1': {1: {'2015-01-05': {'radiation_id': ['1.1 ARC1', '1.2 ARC2'],
    'radiation_ser': [409100, 409101],
    'num_of_fields': 2,
    'images_taken': 2,
    'image_duration': 9,
    'therapist': ['Boisvert-Huneault Christian'],
    'tot_mu': 579.3810000000001,
    'tot_mucoeff': 200.0,
    'resource': '94',
    'allocated': [15],
    'duration': [43],
    'diagnosis': ['Prostate'],
    'gender': ['Male'],
    'oncologist': ['5246'],
    'age': [71],
    'appt_day_of_week': [2],
    'appt_month': [1],
    'appt_hour': [8],
    'plan': ['96732'],
    'orientation': ['HFS']}},
  2: {'2015-01-06': {'radiation_id': ['1.1 ARC1', '1.2 ARC2'],
    'radiation_ser': [409100, 409101],
    'num_of_fields': 2,
    'images_taken': 0,
    'image_duration': 0,
    'therapist': ['nirula manik'],
    'tot_mu': 579.3810000000001,
    'tot_mucoeff': 200.0,
    'resource': '94',
    'allocated': [15],
    'duration': [13],
    'diagnosis': ['Prostate'],
    'gender': ['Male'],
    'oncologist': ['5246'],